In [ ]:
import pandas as pd
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn import preprocessing
import random as rand
import numpy as np
import math

In [ ]:
data = pd.read_csv('games_clean.csv')
labels_num = ['Presence','Memory','ReleaseDate','OriginalCost','DiscountedCost','Achievements','Storage',
                'RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip',
              'Metacritic']

In [ ]:
le = preprocessing.LabelEncoder()
data.loc[data['Metacritic'].notna(),'Metacritic'] = le.fit_transform(data[data['Metacritic'].notna()]['Metacritic'])
dataNum = data[labels_num]
dataNum

In [ ]:
new_labels_num = ['Presence','Memory','ReleaseDate','OriginalCost','DiscountedCost','Achievements','Storage',
                'RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip',
              'Metacritic_0', 'Metacritic_1', 'Metacritic_2', 'Metacritic_3']

dataDummy = pd.get_dummies(dataNum, columns=["Metacritic"])
dataDummy['Metacritic_3'] = [0] * len(dataDummy) # Ninguno tiene 3
dataDummy

In [ ]:
m0 = dataDummy.Metacritic_0 == 0
m1 = dataDummy.Metacritic_1 == 0
m2 = dataDummy.Metacritic_2 == 0
m3 = dataDummy.Metacritic_3 == 0
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_0"] = np.nan
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_1"] = np.nan
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_2"] = np.nan
dataDummy.loc[m0 & m1 & m2 & m3, "Metacritic_3"] = np.nan
dataDummy

In [ ]:
imputer = KNNImputer(n_neighbors=5,weights='uniform')
#dataNumImp = imputer.fit_transform(dataNum)
#dfNum = pd.DataFrame(dataNumImp, columns=labels_num)
dataNumImp = imputer.fit_transform(dataDummy)
dfNum = pd.DataFrame(dataNumImp, columns=new_labels_num)

In [ ]:
for i in range(len(dfNum['ReleaseDate'])):
    dfNum['ReleaseDate'][i] = math.floor(dfNum['ReleaseDate'][i])
    
for i in range(len(dfNum['OriginalCost'])):
    dfNum['OriginalCost'][i] = round(dfNum['OriginalCost'][i], 2)

cols_to_round = ['Achievements', 'RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 
                'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip']
for colunm in cols_to_round:
    for i in range(len(dfNum[colunm])):
        dfNum[colunm][i] = round(dfNum[colunm][i])

In [ ]:
for name in dfNum.columns:
    lis = dfNum[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)
    print('Number of na: ' +str(dfNum[name].isna().sum()))

In [ ]:
dfNum[dfNum.Metacritic_0 >= 0.5].head()

In [ ]:
dfNum[dfNum.Metacritic_1 >= 0.5].head()

In [ ]:
dfNum[dfNum.Metacritic_2 >= 0.5].head()

In [ ]:
X = dfNum.drop(['Metacritic'],axis=1)         # Data
y = dfNum['Metacritic'].astype(int)           # Labels

print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import sklearn.neighbors as nb

cv = StratifiedKFold(n_splits=10) 

cv_scores = cross_val_score(nb.KNeighborsClassifier(), X=X,y=y, cv=cv, scoring='accuracy')  
np.mean(cv_scores)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(dfNum,labels_num)

In [ ]:
dataNum.hist(figsize=(20,20))

In [ ]:
dataNumImp.hist(figsize=(20,20))